In [1]:
import pandas as pd
import requests
import telebot
import yfinance as yf
from datetime import date, datetime
import datetime as dt
from workadays import workdays as wd

In [ ]:
          0      1     2      3      4      5          6          7      8        9       10         11           12      13      14     15    16
       ticker, nada, tipo, modelo, aiotm, strike, dist strike, naosei, preco, negocios, volume, data ultimo, vol implic, delta, gamma, theta, vega, subjacente
["BOVAC12_2024","","CALL",  "E",   "ITM",121.00,     -0.0247,   0.0367, 4.55,   149,   832467.46,"15/02/2024",0.13046874,0.8179,0.0566,-0.0726,9.4582,"BOVA11"]

In [22]:
# Coleta as opções com data de vencimento específica
def opt_venc(ativo, vencimento):
    url = f'https://opcoes.net.br/listaopcoes/completa?idAcao={ativo}&listarVencimentos=false&cotacoes=true&vencimentos={vencimento}'
    r = requests.get(url).json()
    x = ([ativo, vencimento, i[0].split('_')[0], i[2], i[3], i[5], i[8], i[9], i[10], i[11]] for i in r['data']['cotacoesOpcoes'])
    df = pd.DataFrame(x, columns=['ativo', 'vencimento', 'ticker', 'tipo', 'modelo', 'strike', 'preco', 'negocios', 'volume', 'data ult'])
    return df

# Coleta as opções com todos os vencimentos
def opt_all(ativo):
    url = f'https://opcoes.net.br/listaopcoes/completa?idLista=ML&idAcao={ativo}&listarVencimentos=true&cotacoes=true'
    r = requests.get(url).json()
    vencimentos = [i['value'] for i in r['data']['vencimentos']]
    df = pd.concat([opt_venc(ativo, vencimento) for vencimento in vencimentos])
    return df

# Operação - Collar de Alta
def collar_alta(ativo, vencimento, quantidade = 1):
    # Coleta selic anual direto pelo BC
    url_selic = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json'
    selic = pd.read_json(url_selic)
    selic['data'] = pd.to_datetime(selic['data'], dayfirst=True)
    selic.set_index('data', inplace=True)
    selic = float(selic.iloc[-1].values)

    # Calcula a quantidade de dias úteis e o rendimento do CDI até o vencimento
    data_hoje = dt.date.today()
    data_vencimento = datetime.strptime(vencimento, '%Y-%m-%d')
    dias_uteis = wd.networkdays(data_hoje, data_vencimento.date())
    cdi_operacao = round(((1 + selic / 100) ** (dias_uteis / 252) - 1) * 100, 2)

    # Coleta o preço do ativo com base no último fechamento
    preco_ativo = round(yf.download(ativo +'.SA', period='1d')['Adj Close'].iloc[-1], 2)
    
    # Coleta as opções disponíveis para o ativo com base no vencimento determinado
    df = opt_venc(ativo, vencimento)
    
    # Cria um dataframe somente com as PUTs
    df_put = df[df['tipo'] == 'PUT']

    # Cria um dataframe somente com as CALLs e renomeia colunas para diferenciar
    df_call = df[df['tipo'] == 'CALL']
    df_call_op = df_call.copy()
    df_call_op.drop(columns=['vencimento'], inplace=True)
    df_call_op.rename(columns={
        'ticker': 'ticker_call',
        'tipo': 'tipo_call',
        'modelo': 'modelo_call',
        'strike': 'strike_call', 
        'preco': 'preco_call',
        'negocios': 'negocios_call',
        'volume': 'volume_call',
        'data ult': 'data ult_call'
    }, inplace=True)

    # Cria um dataframe com as operações possíveis para cada PUT
    df = pd.merge(df_put, df_call_op, on='ativo', suffixes=('', '_call'))
    df = df[df['strike'] > preco_ativo]
    df['preco ativo'] = preco_ativo
    df['custo'] = (df['preco'] - df['preco_call'] + preco_ativo) * quantidade
    df['cdi oper'] = cdi_operacao
    df['lucro minimo'] = df['strike'] * quantidade - df['custo']
    df['lucro min pct'] = round(df['lucro minimo'] / df['custo'] * 100, 2)
    df['lucro maximo'] = df['strike_call'] * quantidade - df['custo']
    df['lucro max pct'] = round(df['lucro maximo'] / df['custo'] * 100, 2)
    
    # Filtra o dataframe somente com as operações lucrativas
    df_op = df.copy()
    df_op = df_op[df_op['lucro min pct'] >= cdi_operacao]
    df_op = df_op[df_op['lucro maximo'] > 0]
    df_op = df_op[df_op['strike_call'] > df_op['strike']]
    df_op = df_op.sort_values(by='lucro min pct',ascending=True)
    
    return df, df_put, df_call, df_op


In [5]:
ativo = 'BOVA11'
vencimento = '2024-03-15'      # YYYY-MM-DD
qtd = 10


In [23]:
opt_venc(ativo, vencimento)

,ativo,vencimento,ticker,tipo,modelo,strike,preco,negocios,volume,data ult
0,BOVA11,2024-03-15,BOVAC12,CALL,E,121.0,4.55,149.0,832467.46,15/02/2024
1,BOVA11,2024-03-15,BOVAC15,CALL,A,150.0,0.01,2.0,2.99,14/02/2024
2,BOVA11,2024-03-15,BOVAC50,CALL,A,50.0,73.85,1.0,4431.00,15/02/2024
3,BOVA11,2024-03-15,BOVAC91,CALL,A,91.0,35.25,2.0,1797.80,26/01/2024
4,BOVA11,2024-03-15,BOVAC95,CALL,A,102.0,22.66,8.0,3033215.00,15/02/2024
...,...,...,...,...,...,...,...,...,...,...
143,BOVA11,2024-03-15,BOVAO950,PUT,E,95.0,0.01,1108.0,2207.81,15/02/2024
144,BOVA11,2024-03-15,BOVAO960,PUT,E,96.0,0.03,1.0,150.00,15/02/2024
145,BOVA11,2024-03-15,BOVAO970,PUT,E,97.0,0.03,2.0,49.35,15/02/2024
146,BOVA11,2024-03-15,BOVAO980,PUT,E,98.0,0.03,2.0,1300.00,15/02/2024


In [59]:
pd.set_option('display.max_rows', None)

In [6]:
df, df_put, df_call, df_op = collar_alta(ativo, vencimento, qtd)
df_op

ValueError: Expected object or value

In [30]:
df_op.loc[df_op['ativo'] == 'BOVAO140']

,subjacente,vencimento,ativo,tipo,modelo,strike,preco,negocios,volume,ativo_call,...,preco_call,negocios_call,volume_call,preco ativo,custo,cdi oper,lucro minimo,lucro min pct,lucro maximo,lucro max pct


In [20]:
mensagem = f'OPÇÕES DE HOJE: \n{ativo}'
bot = telebot.TeleBot("6832366281:AAF8pcibmxZGWyPUeGHgHOT64qkMgiKGCQA")
grupo = "-1001803272086"
bot.send_message(grupo, mensagem)